# Course 27666 AI-guided Protein Science

# Graph Machine Learning

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Multiomics-Analytics-Group/course_graph_machine_learning/blob/main/notebooks/graph_level_prediction.ipynb)

# HIV Inhibitor Prediction

## A Graph-level Task

Human immunodeficiency virus type 1 (HIV-1) is the most common cause of Acquired Immunodeficiency Syndrome (AIDS). One ongoing area of research is finding compounds that inhibit HIV-1 viral replication.

![HIV](https://ars.els-cdn.com/content/image/1-s2.0-S0968089619306704-ga1.jpg)



Using dataaset from: [dgl](https://www.dgl.ai/)

In [ ]:
!pip install networkx graphein pandas biopython rdkit openpyxl pyvis
!pip install torch torch-geometric matplotlib

In [89]:
import pandas as pd
import graphein.molecule as gm
from functools import partial
import networkx as nx
from torch_geometric.data import Data, DataLoader
from torch.utils.data import random_split
import torch
from torch_geometric.nn.conv import GCNConv
from torch_geometric.nn.pool import global_mean_pool
import torch.nn.functional as F

In [90]:
hiv_dataset = pd.read_csv('https://raw.githubusercontent.com/Multiomics-Analytics-Group/course_graph_machine_learning/refs/heads/main/data/hiv_dgl.tsv', sep='\t')

In [ ]:
hiv_dataset.head()

## Using SMILES Graph to Predict HIV Inhibition

We will make use of [Graphein](https://graphein.ai/) to convert SMILES into a Molecular Graph making sure to collect molecular features that can be incorporated to the prediction model.

In [93]:
config = gm.MoleculeGraphConfig(
    node_metadata_functions=[
        gm.atom_type_one_hot,
        gm.atomic_mass,
        gm.degree,
        gm.total_degree,
        gm.total_valence,
        gm.explicit_valence,
        gm.implicit_valence,
        gm.num_explicit_h,
        gm.num_implicit_h,
        gm.total_num_h,
        gm.num_radical_electrons,
        gm.formal_charge,
        gm.hybridization,
        gm.is_aromatic,
        gm.is_isotope,
        gm.is_ring,
        gm.chiral_tag,
        partial(gm.is_ring_size, ring_size=5),
        partial(gm.is_ring_size, ring_size=7)
    ]
)


In [ ]:
graphs = []
labels = []
for index,row in hiv_dataset[['smiles','HIV_active']].iterrows():
    try:
        graph = gm.construct_graph(smiles=row["smiles"], config=config)
        graphs.append(graph)
        labels.append(row["HIV_active"])
    except:
        pass        

### Multiple Graphs

In [ ]:
graphs[0].nodes(data=True)

### Converting to PyG Datasets

In [82]:
pygraphs = []
for i, g in enumerate(graphs):
    mapping = {n:int(n.split(":")[1]) for n in g.nodes()}
    g = nx.relabel_nodes(g, mapping)
    source_nodes = [n1 for n1,n2 in list(g.edges())]
    target_nodes = [n2 for n1,n2 in list(g.edges())]
    edge_indices = torch.tensor([source_nodes,
                                 target_nodes],
                                dtype=torch.long)
    atom_features = torch.tensor([[g.nodes[node]["atomic_num"]] for node in g.nodes()], dtype=torch.float)
    # Create graph data
    data = Data(x=atom_features, edge_index=edge_indices)
    pygraphs.append(data)

### Adding the Labels HIV_activity

In [83]:
for i, g in enumerate(pygraphs):
    g.y = torch.tensor([labels[i]], dtype=torch.long)

### Splitting into Training and Test

In [96]:
# Split dataset into training and test sets
train_size = int(0.7 * len(graphs))
test_size = len(graphs) - train_size
train_dataset, test_dataset = random_split(pygraphs, [train_size, test_size])

### Create Dataloaders

Memory efficient.

In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Define GNN Model

In [97]:
class GNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = torch.nn.Linear(hidden_dim, output_dim) # maps the learned graph-level feature representations to the output space, enabling prediction tasks.

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch) # Essential for aggregating node-level features into a single graph-level feature vector.
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

# Initialize model, loss, and optimizer
model = GNN(input_dim=1, hidden_dim=16, output_dim=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Training Loop

In [ ]:
for epoch in range(10):
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

### Model Evaluation

In [ ]:
model.eval()
correct = 0
for data in test_loader:
    out = model(data.x, data.edge_index, data.batch)
    pred = out.argmax(dim=1)
    correct += (pred == data.y).sum().item()

accuracy = correct / len(test_dataset)
print(f"Test Accuracy: {accuracy:.2f}")